<a href="https://colab.research.google.com/github/felipeeduardor/SQL-integrado-com-Python-/blob/main/SQL_e_Psycopg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Usando BD e SQL internamente no Python**

# **Instalando um Bancos de Dados**

In [1]:
!sudo apt-get -y install postgresql postgresql-contrib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common postgresql-contrib ssl-cert
  sysstat
0 upgraded, 14 newly installed, 0 to remove and 29 not upgraded.
Need to get 18.4 MB of archives.
After this operation, 52.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com

# **Verifica se foi instalado corretamente**

In [2]:
!service postgresql status

14/main (port 5432): down


# **caso down, stop e start novamente**

In [3]:
!service postgresql stop

 * Stopping PostgreSQL 14 database server
   ...done.


In [4]:
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [5]:
!service postgresql status

14/main (port 5432): online


# **Criação de Usuário e Senha**

In [6]:
# Conectar ao PostgreSQL
!sudo -u postgres psql -c "CREATE USER cdpro WITH PASSWORD 'admincdpro';"

CREATE ROLE


# **Criação do BD**

In [7]:
!sudo -u postgres psql -c "CREATE DATABASE clientes OWNER cdpro;"

CREATE DATABASE


# **Acessando e usando o BD com pacote Psycopg2**


# **Conexão com o BD**

In [8]:
import psycopg2

dbname = 'clientes'
user = 'cdpro'
password = 'admincdpro'
host = 'localhost'

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    print("Conexão bem-sucedida!")
except psycopg2.Error as e:
    print("Erro ao conectar ao banco de dados:", e)



Conexão bem-sucedida!


# **Acessando uma tabela que será gravada no BD**

In [9]:
import pandas as pd

In [12]:
df = pd.read_csv('/content/clientes.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cod_cliente           614 non-null    object 
 1   sexo                  601 non-null    object 
 2   estado_civil          611 non-null    object 
 3   dependentes           599 non-null    object 
 4   educacao              614 non-null    object 
 5   empregado             582 non-null    object 
 6   renda                 614 non-null    int64  
 7   renda_conjuge         614 non-null    object 
 8   emprestimo            592 non-null    float64
 9   prestacao_mensal      600 non-null    float64
 10  historico_credito     564 non-null    float64
 11  imovel                614 non-null    object 
 12  aprovacao_emprestimo  614 non-null    object 
dtypes: float64(3), int64(1), object(9)
memory usage: 62.5+ KB


Salvando a tabela no BD

In [13]:
import psycopg2

# Conectar ao banco de dados
dbname = 'clientes'
user = 'cdpro'
password = 'admincdpro'
host = 'localhost'

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    print("Conexão bem-sucedida!")
except psycopg2.Error as e:
    print("Erro ao conectar ao banco de dados:", e)

# Criar a tabela 'clientes' se ela não existir
try:
    cursor = conn.cursor()
    create_table_query = '''CREATE TABLE IF NOT EXISTS clientes (
                            cod_cliente VARCHAR(100),
                            sexo VARCHAR(100),
                            estado_civil VARCHAR(100),
                            dependentes VARCHAR(100),
                            educacao VARCHAR(100),
                            empregado VARCHAR(100),
                            renda INTEGER,
                            renda_conjuge VARCHAR(100),
                            emprestimo FLOAT,
                            prestacao_mensal FLOAT,
                            historico_credito FLOAT,
                            imovel VARCHAR(100),
                            aprovacao_emprestimo VARCHAR(100)
                            );'''
    cursor.execute(create_table_query)
    conn.commit()
    print("Tabela 'clientes' criada com sucesso!")

except psycopg2.Error as e:
    conn.rollback()  # Desfaz a transação em caso de erro
    print("Erro ao criar tabela:", e)


Conexão bem-sucedida!
Tabela 'clientes' criada com sucesso!


In [15]:
import pandas as pd
from io import StringIO

# Carregar os dados do DataFrame Pandas (supondo que o DataFrame já foi criado)
df = pd.read_csv('/content/clientes.csv')

# Ajustar os tipos de dados no DataFrame antes de carregar no banco de dados
df['renda'] = df['renda'].astype(int)  # Convertendo para tipo inteiro
df['emprestimo'] = df['emprestimo'].astype(float)  # Convertendo para tipo float
df['prestacao_mensal'] = df['prestacao_mensal'].astype(float)  # Convertendo para tipo float
df['historico_credito'] = df['historico_credito'].astype(float)  # Convertendo para tipo float

In [16]:
df

,cod_cliente,sexo,estado_civil,dependentes,educacao,empregado,renda,renda_conjuge,emprestimo,prestacao_mensal,historico_credito,imovel,aprovacao_emprestimo
0,LP001002,Male,No,0,Graduate,No,5849,0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0,187.0,360.0,1.0,Urban,Y


In [17]:
import psycopg2

# Criar um buffer de memória para os dados do DataFrame
buffer = StringIO()
df.to_csv(buffer, index=False, header=False)  # Não inclui o cabeçalho no arquivo CSV

# Carregar os dados do buffer diretamente para o banco de dados
try:
    cursor = conn.cursor()
    buffer.seek(0)
    cursor.copy_from(buffer, 'clientes', sep=',', columns=df.columns, null='')  # null='' para tratar valores nulos
    conn.commit()
    print("Dados carregados com sucesso!")
except psycopg2.Error as e:
    conn.rollback()  # Desfaz a transação em caso de erro
    print("Erro ao carregar dados:", e)
finally:
    buffer.close()


Dados carregados com sucesso!


# **Usando o Python/Psycopg2**

In [18]:
# Consulta SQL para calcular a soma da renda por educação na tabela 'clientes'
sql_query = """

SELECT

        estado_civil,

        sum(renda) AS renda_total



FROM    clientes


GROUP BY
        estado_civil;

"""

In [19]:
# Função para executar consultas SQL e retornar um DataFrame


def execute_query_to_dataframe(query):

    df = pd.read_sql_query(query, conn)



    return df

In [20]:
# Executar a consulta SQL e exibir o resultado
result_df = execute_query_to_dataframe(sql_query)


print("Resultado da consulta:")



print(result_df)

Resultado da consulta:
  estado_civil  renda_total
0         None        18621
1          Yes      2240411
2           No      1058692


<ipython-input-19-c13185c836cf>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [21]:
result_df

,estado_civil,renda_total
0,None,18621
1,Yes,2240411
2,No,1058692


# **Contagem de Registros da Tabela**

In [22]:
# Consulta SQL para calcular a soma da renda por educação na tabela 'clientes'
sql_query = """
SELECT

count(*) as QTD_REG

FROM clientes

;
"""

In [23]:
# Função para executar consultas SQL e retornar um DataFrame
def execute_query_to_dataframe(query):
    df = pd.read_sql_query(query, conn)
    return df

In [24]:
# Executar a consulta SQL e exibir o resultado
result_df = execute_query_to_dataframe(sql_query)


print("Resultado da consulta:")



print(result_df)

Resultado da consulta:
   qtd_reg
0      614


<ipython-input-23-9c1339bee8f2>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [25]:
result_df

,qtd_reg
0,614


# **Query 3**

In [26]:
# Consulta SQL para calcular a soma da renda por educação na tabela 'clientes'
sql_query = """
SELECT
    estado_civil AS Estado_Civil,
    COUNT(*) AS Total_Clientes,
    AVG(renda) AS Media_Renda,
    MAX(renda) AS Maior_Renda,
    MIN(renda) AS Menor_Renda,
    SUM(CASE WHEN aprovacao_emprestimo = 'Aprovado' THEN 1 ELSE 0 END) AS Total_Aprovados,
    SUM(CASE WHEN aprovacao_emprestimo = 'Reprovado' THEN 1 ELSE 0 END) AS Total_Reprovados
FROM
    clientes
GROUP BY
    estado_civil;


"""

In [27]:
# Função para executar consultas SQL e retornar um DataFrame
def execute_query_to_dataframe(query):
    df = pd.read_sql_query(query, conn)
    return df

In [28]:
# Executar a consulta SQL e exibir o resultado
result_df = execute_query_to_dataframe(sql_query)


print("Resultado da consulta:")



print(result_df)

Resultado da consulta:
  estado_civil  total_clientes  media_renda  maior_renda  menor_renda  \
0         None               3  6207.000000        10047         3816   
1           No             213  4970.384977        37719          416   
2          Yes             398  5629.173367        81000          150   

   total_aprovados  total_reprovados  
0                0                 0  
1                0                 0  
2                0                 0  


<ipython-input-27-9c1339bee8f2>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [29]:
result_df

,estado_civil,total_clientes,media_renda,maior_renda,menor_renda,total_aprovados,total_reprovados
0,None,3,6207.000000,10047,3816,0,0
1,No,213,4970.384977,37719,416,0,0
2,Yes,398,5629.173367,81000,150,0,0


# **Download do Banco**

In [30]:
%env PGHOST=localhost       # Postgree Host
%env PGUSER=cdpro           # Pstgree User
%env PGPASSWORD=admincdpro  # ...
%env PGDATABASE=clientes    # ...


env: PGHOST=localhost       # Postgree Host
env: PGUSER=cdpro           # Pstgree User
env: PGPASSWORD=admincdpro  # ...
env: PGDATABASE=clientes    # ...


In [31]:
# -h (host)
# -U (user)
# -d (Database)

!pg_dump -h $PGHOST -U $PGUSER -d $PGDATABASE > backup_clientes.sql

pg_dump: error: too many command-line arguments (first is "#")
Try "pg_dump --help" for more information.


# **FIM**